In [ ]:
!pip install kaggle
!pip install geopandas
!pip install missingno
!pip install shapely
!pip install mpl_toolkits
!pip install sklearn
!pip install seaborn
!pip install matplotlib
!pip install numpy
!pip install pandas

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import missingno as msno
import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shapely.geometry import Point
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# from google.colab import files
# files.upload()
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# !kaggle datasets download -d START-UMD/gtd
# !unzip gtd.zip -d gtd_dataset
# import pandas as pd
# df = pd.read_csv('gtd_dataset/globalterrorismdb_0718dist.csv')
# print(df.head())

In [ ]:
df = pd.read_csv('Global_Terrorism_DB.csv', encoding='latin-1')

print("Dataset Info:")
print(df.info())
print("\nFirst Few Rows:")
print(df.head())

<ipython-input-7-273a4eb19791>:1: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Global_Terrorism_DB.csv', encoding='latin-1')


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181691 entries, 0 to 181690
Columns: 135 entries, eventid to related
dtypes: float64(55), int64(22), object(58)
memory usage: 187.1+ MB
None

First Few Rows:
        eventid  iyear  imonth  iday approxdate  extended resolution  country  \
0  197000000001   1970       7     2        NaN         0        NaN       58   
1  197000000002   1970       0     0        NaN         0        NaN      130   
2  197001000001   1970       1     0        NaN         0        NaN      160   
3  197001000002   1970       1     0        NaN         0        NaN       78   
4  197001000003   1970       1     0        NaN         0        NaN      101   

          country_txt  region  ... addnotes scite1 scite2  scite3  dbsource  \
0  Dominican Republic       2  ...      NaN    NaN    NaN     NaN      PGIS   
1              Mexico       1  ...      NaN    NaN    NaN     NaN      PGIS   
2         Philippines       5  ...      NaN    NaN    

In [ ]:
print(df.columns.tolist())

['eventid', 'iyear', 'imonth', 'iday', 'approxdate', 'extended', 'resolution', 'country', 'country_txt', 'region', 'region_txt', 'provstate', 'city', 'latitude', 'longitude', 'specificity', 'vicinity', 'location', 'summary', 'crit1', 'crit2', 'crit3', 'doubtterr', 'alternative', 'alternative_txt', 'multiple', 'success', 'suicide', 'attacktype1', 'attacktype1_txt', 'attacktype2', 'attacktype2_txt', 'attacktype3', 'attacktype3_txt', 'targtype1', 'targtype1_txt', 'targsubtype1', 'targsubtype1_txt', 'corp1', 'target1', 'natlty1', 'natlty1_txt', 'targtype2', 'targtype2_txt', 'targsubtype2', 'targsubtype2_txt', 'corp2', 'target2', 'natlty2', 'natlty2_txt', 'targtype3', 'targtype3_txt', 'targsubtype3', 'targsubtype3_txt', 'corp3', 'target3', 'natlty3', 'natlty3_txt', 'gname', 'gsubname', 'gname2', 'gsubname2', 'gname3', 'gsubname3', 'motive', 'guncertain1', 'guncertain2', 'guncertain3', 'individual', 'nperps', 'nperpcap', 'claimed', 'claimmode', 'claimmode_txt', 'claim2', 'claimmode2', 'claim

In [ ]:
duplicates = df.duplicated().sum()
print("\nNumber of Duplicates:")
print(duplicates)
df.drop_duplicates(inplace=True)


Number of Duplicates:
0


In [ ]:
null_cells = df.isnull().sum()
print("\nNull Cells:")
print(null_cells[null_cells > 0])

missing_values = df.isnull().sum()
print("\nMissing Values:")
print(missing_values[missing_values > 0])
df.fillna('Unknown', inplace=True)


Null Cells:
approxdate    172452
resolution    179471
provstate        421
city             435
latitude        4556
               ...  
addnotes      153402
scite1         66191
scite2        104758
scite3        138175
related       156653
Length: 106, dtype: int64

Missing Values:
approxdate    172452
resolution    179471
provstate        421
city             435
latitude        4556
               ...  
addnotes      153402
scite1         66191
scite2        104758
scite3        138175
related       156653
Length: 106, dtype: int64


<ipython-input-9-4f90dcd2b568>:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('Unknown', inplace=True)


In [ ]:
df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]

In [ ]:
print("Data Types:")
print(df.dtypes)

Data Types:
eventid        int64
iyear          int64
imonth         int64
iday           int64
approxdate    object
               ...  
int_log        int64
int_ideo       int64
int_misc       int64
int_any        int64
related       object
Length: 135, dtype: object


In [ ]:
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns
print("Numerical Columns:")
print(numerical_columns)

Numerical Columns:
Index(['eventid', 'iyear', 'imonth', 'iday', 'extended', 'country', 'region',
       'vicinity', 'crit1', 'crit2', 'crit3', 'success', 'suicide',
       'attacktype1', 'targtype1', 'individual', 'weaptype1', 'property',
       'int_log', 'int_ideo', 'int_misc', 'int_any'],
      dtype='object')


In [ ]:
scaler = MinMaxScaler()
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])
print("Normalized numerical columns:")
print(df.head())

Normalized numerical columns:
        eventid  iyear    imonth      iday approxdate  extended resolution  \
0  0.000000e+00    0.0  0.583333  0.064516    Unknown       0.0    Unknown   
1  2.122107e-10    0.0  0.000000  0.000000    Unknown       0.0    Unknown   
2  2.122101e-04    0.0  0.083333  0.000000    Unknown       0.0    Unknown   
3  2.122104e-04    0.0  0.083333  0.000000    Unknown       0.0    Unknown   
4  2.122106e-04    0.0  0.083333  0.000000    Unknown       0.0    Unknown   

   country         country_txt    region  ... addnotes   scite1   scite2  \
0    0.054  Dominican Republic  0.090909  ...  Unknown  Unknown  Unknown   
1    0.126              Mexico  0.000000  ...  Unknown  Unknown  Unknown   
2    0.156         Philippines  0.363636  ...  Unknown  Unknown  Unknown   
3    0.074              Greece  0.636364  ...  Unknown  Unknown  Unknown   
4    0.097               Japan  0.272727  ...  Unknown  Unknown  Unknown   

    scite3 dbsource int_log  int_ideo int_mi

In [ ]:
standard_scaler = StandardScaler()
df[numerical_columns] = standard_scaler.fit_transform(df[numerical_columns])
print("Standardized numerical columns:")
print(df.head())

Standardized numerical columns:
    eventid     iyear    imonth      iday approxdate  extended resolution  \
0 -2.466545 -2.461575  0.157225 -1.532291    Unknown -0.217945    Unknown   
1 -2.466545 -2.461575 -1.908712 -1.759202    Unknown -0.217945    Unknown   
2 -2.465791 -2.461575 -1.613578 -1.759202    Unknown -0.217945    Unknown   
3 -2.465791 -2.461575 -1.613578 -1.759202    Unknown -0.217945    Unknown   
4 -2.465791 -2.461575 -1.613578 -1.759202    Unknown -0.217945    Unknown   

    country         country_txt    region  ... addnotes   scite1   scite2  \
0 -0.657999  Dominican Republic -1.759371  ...  Unknown  Unknown  Unknown   
1 -0.017511              Mexico -2.100272  ...  Unknown  Unknown  Unknown   
2  0.249359         Philippines -0.736667  ...  Unknown  Unknown  Unknown   
3 -0.480086              Greece  0.286037  ...  Unknown  Unknown  Unknown   
4 -0.275486               Japan -1.077568  ...  Unknown  Unknown  Unknown   

    scite3 dbsource   int_log  int_ideo  i

In [ ]:
categorical_columns = [
    'country', 'region', 'provstate', 'city', 'attacktype1_txt',
    'targtype1_txt', 'targsubtype1_txt', 'gname', 'claimmode_txt',
    'weaptype1_txt', 'weapsubtype1_txt', 'propextent_txt', 'hostkidoutcome_txt'
]

df_encoded = pd.get_dummies(df, columns=categorical_columns)
print("One-hot encoded columns:")
print(df_encoded.head())

In [ ]:
standard_scaler = StandardScaler()
df[numerical_columns] = standard_scaler.fit_transform(df[numerical_columns])
print("Standardized numerical columns:")
print(df.head())

In [ ]:
print("Summary Statistics:")
print(df.describe())
print("\nCategorical Columns Summary:")
print(df.select_dtypes(include=['object']).describe())

In [ ]:
df['iyear'].value_counts().sort_index().plot(kind='bar')
plt.title('Number of Attacks Per Year')
plt.xlabel('Year')
plt.ylabel('Number of Attacks')
plt.show()

In [ ]:
df['country_txt'].value_counts().head(10).plot(kind='bar')
plt.title('Top 10 Countries by Number of Attacks')
plt.xlabel('Country')
plt.ylabel('Number of Attacks')
plt.show()

In [ ]:
correlation_matrix = df.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
msno.matrix(df)
plt.show()

In [ ]:
df['attacktype1_txt'].value_counts().plot(kind='bar')
plt.title('Distribution of Attack Types')
plt.xlabel('Attack Type')
plt.ylabel('Frequency')
plt.show()

In [ ]:
df[['nkill', 'nwound']].fillna(0).sum().plot(kind='bar')
plt.title('Total Casualties')
plt.xlabel('Casualty Type')
plt.ylabel('Number of Casualties')
plt.show()

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

fig, ax = plt.subplots(1, 1, figsize=(15, 10))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

world.boundary.plot(ax=ax, linewidth=1)
gdf.plot(ax=ax, markersize=10, color='red', alpha=0.5)
plt.title('Geographical Distribution of Terrorism Events')
plt.show()

In [ ]:
df['motive'].value_counts().head(10).plot(kind='bar')
plt.title('Top 10 Motives')
plt.xlabel('Motive')
plt.ylabel('Frequency')
plt.show()

In [ ]:
df['gname'].value_counts().head(10).plot(kind='bar')
plt.title('Top 10 Terrorist Groups')
plt.xlabel('Group Name')
plt.ylabel('Frequency')
plt.show()